## Example for extending Mondrian Trees

In [1]:
import Base.copy
using Distributions
using MLBase
using Plots
include("./Mondrian_Forest_Classifier.jl"); 
# include("Mondrian_extention.jl")

LoadError: LoadError: LoadError: [91mUndefVarError: Mondrian_Tree_Classifier not defined[39m
while loading /home/annika/RSG Julia/warwick-rsg/MondrianForest/Mondrian_extention.jl, in expression starting on line 132
while loading /home/annika/RSG Julia/warwick-rsg/MondrianForest/Mondrian_Forest_Classifier.jl, in expression starting on line 2

In [ ]:
"""
`function expand_forest!(MF::Mondrian_Forest_Classifier,X_extend, Y_extend,λ)`

This function expands an already sampled Mondrian Forest Classifier by a desired number of datapoints.

`Input`: Mondrian Forest Classifier MF (abstract type Mondrian_Forest_Classifier), array of features X_extend to extend the forest on, Array of class labels Y_extend corresponding to the new features, life time parameter λ

`Output`: Mondrian Forest Classifier with incoporated new datapoints

`Files needed to run this function`: Mondrian_Forest_Classifier.jl", "Mondrian_extention.jl"

This function calls the function expand!. 
"""

In [ ]:
"""
`function expand!(T::Mondrian_Tree_Classifier,X::Array{Float64,N} where N,Y::Array{Int64},λ::Float64)`

This function expands an already sampled Mondrian Tree Classifier by a desired number of datapoints. 

`Input`: Mondrian Tree Classifier T (abstract type Mondrian_Tree_Classifier), array of features X (Array of Float64), array of class labels (1dim of Float 64), Lifetime parameter λ (Float 64)

Each row in the array X represents one set of features, the corresponding row in Y represents the class label. 

`Output`: Mondrian Tree Classifier with incoporated new datapoints

`Files needed to run this function`: Mondrian_Forest_Classifier.jl", "Mondrian_extention.jl"

This function calls the function Extend_Mondrian_Tree. 
"""

This notebook explains how to extend Mondrian Trees and Mondrian Forests. Using an online algorithm, we can incorporate data points one by one.

### Functions

All functions used in this section can be found in the files "Mondrian_extention.jl" and "Mondrian_extention_utils.jl". 

In [ ]:
?Extend_Mondrian_Tree!

In [ ]:
?Extend_Mondrian_Tree!

In [ ]:
?expand!

In [ ]:
?expand_forest!

#### 1) Prepare some data

We generate two sets of fakedata (features + classes). We use the first one to train on and the second one to extend on.

In [ ]:
function Fakedata3Classif(N,d,N_test=0)
    X = randn((N,d))
    param1 = randn(d)
    param2 = randn(d)
    Y = ( sum(X*param1,2) .> mean(sum(X*param2,2)) )
    Y=convert(Array{Int},Y)
    for i in 1:size(X,1)
        if X[i,2] > 1.5
            Y[i] = 3
        end
    end
    if (N_test > 0)
        x = randn((N_test,d))
        y = ( sum(x*param1,2) .> mean(sum(x*param2,2)) )
        y=convert(Array{Int},y)
        for i in 1:size(x,1)
            if x[i,2] > 1.5
                y[i] = 3
            end
        end
        return X,Y,x,y
    end
    return X,Y
end

In [ ]:
X, Y, x, y = Fakedata3Classif(1000,2,1000);  #X,Y: train set ; x,y: extention data
Y = Y.+1;
y = y.+1;

In [ ]:
if size(X,2)==2
    plot()
    for class in unique(Y)
        scatter!(X[Y.==class,1],X[Y.==class,2],label="$class")
    end
    plot!()
end

### 1) Extend a single tree

To extend a single tree, we can just use the function expand!. We train a Mondrian Tree Classifier on the dataset X,Y and then extend it on the dataset x,y which we created above. We compute the training accuracy of the extended tree. For comparison, we then train a Mondrian Tree Classifier on the merges dataset X,x Y,y (it was then trained on all of the data the extended tree was trained on, without using extentions) and compute the training accuracy as well.  

In [ ]:
T = Mondrian_Tree_Classifier()
train!(T,X,Y,1e9);  #Sample tree on training dataset

In [ ]:
expand!(T,x,y,1e9);  #Expand the tree on the extention dataset

In [ ]:
#merge training and extention data to get the total data the tree was trained on

All_data=vcat(X,x);             
all_labels = vcat(Y,y);

In [ ]:
#let the extended tree predict the training data, compute training accuracy
predict!(T,All_data);     
println("Train Accuracy of extended tree")
println(correctrate(all_labels,convert(Array{Int,1},pred)))

In [ ]:
#compute a tree on the merged dataset (classic, no extention)
T_full = Mondrian_Tree_Classifier()
train!(T_full,All_data,all_labels,1e9);
# T_full = Sample_Mondrian_Tree!(T,1e9,All_data,all_labels);  #compute tree of the merged dataset

In [ ]:
#let the extended tree predict the training data, compute training accuracy

println("Train Accuracy tree trained on whole dataset")
predict!(T_full,All_data)
println(correctrate(all_labels,convert(Array{Int,1},pred)))

In [ ]:
# function expand!(T::Mondrian_Tree_Classifier,X::Array{Float64,N} where N,Y::Array{Int64},λ::Float64)    
    
#     # puts the extention in a nice framework, allows to extend by multiple datapoints
    
#     for i=1:length(X[:,1])
#         T.Tree=Extend_Mondrian_Tree!(T.Tree,λ,X[i,:],Y[i]);
#     end
#     T.X = vcat(T.X,X)
#     T.Y = vcat(T.Y,Y)
#     compute_predictive_posterior_distribution!(T.Tree,10*size(X,2))
#     return T    
# end

We see that the training accuracy is the same weather we extend the tree or weather we sample the tree on the whole dataset immediatly. This proves that the extention implementation works.

### 2) Extend Mondrian Forests

Now we extend a Mondrian Forest classifier that was pretrained on the  training same dataset (X,Y) as above. We again extend the classifier and then train a new classifier on the whole datset for comparison.

In [ ]:
# function expand_forest!(MF::Mondrian_Forest_Classifier,X_extend, Y_extend,λ)
#     X=MF.X
#     Y=MF.Y
#     if size(X)[2] != size(X_extend)[2]
#         println("Error - the number of features in the new data doesn't fit the original data")
#     end
#     Trees=MF.Trees
#     for i=1:MF.n_trees
#         T = expand!(Trees[i], X_extend,Y_extend,λ)
#         Trees[i]=T
#     end
#      MF.Trees=Trees
#      MF.X=vcat(X,X_extend)
#      MF.Y=vcat(Y,Y_extend)
    
# end

In [ ]:
MF = Mondrian_Forest_Classifier(50)  #initialise Mondrian Forest Classifier with 50 Trees

In [ ]:
train!(MF, X, Y, 1e9);  #Train the classifier on the same dataset as before

In [ ]:
expand_forest!(MF,x,y,1e9);  #expand the forest on the dataset x,y

In [ ]:
MF_control = Mondrian_Forest_Classifier(50)   #initialise control classifier with 50 Trees
train!(MF_control, All_data, all_labels, 1e9);  # train control classifier on merged dataset

In [ ]:
#compute train accuracy for extended forest

pred=predict!(MF_control, All_data);
println("Train Accuracy")
println(correctrate(all_labels,convert(Array{Int,1},pred)))

In [ ]:
#compute train accuracy for classic forest

pred = predict!(MF,All_data);
println("Train Accuracy")
println(correctrate(all_labels,convert(Array{Int,1},pred)))

#### Some Testing and Benchmarking

How fast is the extention? 